In [1]:
import numpy as np
from random import randint
from math import floor, fabs
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys

def SecModelParameter ( VarsValue, m, n, my_rhs, my_coef, BranchedVariables):
#     print 'BranchedVariables',BranchedVariables
    unbranched=list()
    c=[]
    d=(np.zeros(((2 * m) + (4 * n), 2*n)))
    halatest=[]
    a=[]
    newrow=[]
    zer=(np.zeros(((2 * m) + (4 * n), 2*n)))
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)
    NodeNum=n-len(unbranched)
    
    
    for i in unbranched:   
        for h in range (m):
            c.append(0)
        for h in range (m):
            lin=0            
            for j in range (n):
#                 print 'i', i, 'h', h,'j',j
#                 print 'j1',j
                if j in (BranchedVariables):
#                     print 'j2',j
                    sv = VarsValue[BranchedVariables.index(j)]
#                     print 'sv', sv
                    lin =lin+ (sv * my_coef[h][j]) 
#                     print 'lin', lin
#                 print 'sv'
            lin=lin- my_rhs[h]
            c.append(lin)                
        for h in range (n):
            c.append(0)
        for h in range (n):
            c.append(0)

        for jj in range (n):
            lin2=-1
            if jj in (BranchedVariables):
                sv = VarsValue[BranchedVariables.index(jj)]
                lin2 =lin2+ sv
            c.append(lin2)
            
        for jj in range (n):
            lin2=0
            if jj in (BranchedVariables):
                lin2 =-1*VarsValue[BranchedVariables.index(jj)]
            c.append(lin2)

              
    k=NodeNum-1 

    for i in unbranched: 
        k=k+1
        for t in range(m):              
            for j in range(n):    
                d[(k-NodeNum)*(2*m+4*n)+t][j+n]=my_coef[t][j] 
                
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t][n+i]=d[(k-NodeNum)*(2*m+4*n)+t][n+i]-my_rhs[t]   
                
        for t in range(m):  
            for kk in unbranched:
                d[(k-NodeNum)*(2*m+4*n)+t+m][kk]=my_coef[t][kk] 
                

        for t in range(m):  
            for j in range(n):
                d[(k-NodeNum)*(2*m+4*n)+t+m][j+n]=-1*my_coef[t][j] 
                 
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]=d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]+my_rhs[t] 
                
        for t in range(n):              
            d[(k-NodeNum)*(2*m+4*n)+t+2*m][i]=-1 
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+t][n+t]=1 
        
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+n+t][n+t]=-1 
        
        for mm in unbranched: 
             
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+mm][mm]=1  
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]=d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]+1            
            

        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+t][n+t]=-1 

        for t in unbranched:  
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][t]=-1  
               
        
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][n+t]=1 
            
        d=np.vstack([d, zer])                          
            
    d = np.delete(d, slice(((n - NodeNum)*((2 * m) + (4 * n))),((n-NodeNum+1)*((2 * m) + (4 * n)))), axis=0)
#     print 'c',c
#     print 'unbranched',unbranched
    return c, d,unbranched
#     print c, d,unbranched

In [2]:
def SecondModel (cc,dd,NotBranched):
#     print 'cc,dd,NotBranched',cc,dd,NotBranched
    numrows = len(dd)    # 3 rows in your example
    numcols = len(dd[0])
    VarNum=numcols/2
    ConstNum=numrows/len(NotBranched)
    my_obj=[]
    my_ub=[]
    my_lb=[]
    my_colnames=[]
    my_rownames=["c1"]
    my_coef=[] 
    my_row=[]
    my_rhs=[]
    my_sense=""
    my_Nmah=list()
    
    for i in range(numrows):
        my_obj=cc
        my_ub.append(1)
        my_lb.append(0)
        my_colnames.append("y"+str(i))

    sumarray=[]     
    NotBranched.sort()    
#     sumshode=list()
    for t in range (VarNum):
        sumlist=list()
        for j in range(numrows):
#             print 't,j',t,j
            if t in NotBranched: 
                indj=NotBranched.index(t)
#                 print 'indj',indj
                indjrange=range(indj*ConstNum,(indj+1)*ConstNum)
#                 print 'indjrange',indjrange
                if j in indjrange:
#                     print 'dd[j,t]+dd[j,t+VarNum]',j,'ll',dd[j,t]+dd[j,t+VarNum]
                    sumlist.append(dd[j,t]+dd[j,t+VarNum])
                    dd[j,t+VarNum]=0
                else:
                    sumlist.append(dd[j,t])
            else:
                sumlist.append(dd[j,t])
        sumarray.append(sumlist)
#     print 'sumarray',sumarray
#     print 'newDD',dd
  
#     print 'notbranched',NotBranched
    lenNotbranched=len(NotBranched)
   
    
    for t in range(lenNotbranched):  
        for i in range (VarNum):
            sumlist=list()
            if t>0:
                for p in range(t):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                        
            for j in range(ConstNum):
                sumlist.append(dd[t*ConstNum+j,i+VarNum])
            if i in NotBranched and i > NotBranched[t]:
                indx=NotBranched.index(i)
                indy=NotBranched.index(NotBranched[t])
                
                for x in range(indx-indy-1):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                for j in range(ConstNum):
                    sumlist.append(dd[(indx)*ConstNum+j,NotBranched[t]+VarNum])
                    dd[(indx)*ConstNum+j,NotBranched[t]+VarNum]=0
#                 sumarray.append([t+indx,indy])
                if t<lenNotbranched-2 :
                    for p in range(lenNotbranched-indx-1):
                        for j in range (ConstNum):
                            sumlist.append(0)
            else:
                if t<lenNotbranched-1 :
#                     print 'lenNotbranched',lenNotbranched
                    for p in range(lenNotbranched-t-1):
                            for j in range (ConstNum):
                                sumlist.append(0) 
#             print 't,i',t,',',i
#             print 'sumlist p2',sumlist
#             print 'lensumlist p2',len(sumlist)
            sumarray.append(sumlist)
#     print 'part2' ,sumarray  
    
    rows=[]
    for j in range(numrows):
        my_row.append(1)
    rows.append([range(numrows),my_row])  
    def populatebyrow(SecProb):
        SecProb.objective.set_sense(SecProb.objective.sense.maximize)
        SecProb.variables.add(obj = my_obj,lb=my_lb, names = my_colnames)
        
        for i in range (len(sumarray)):
            if all(k == 0 for k in sumarray[i])!=True:
#                 print 'i',i
                sumline=[]
                sumline.append([range(len(sumarray[i])),sumarray[i]])
#                 print 'sumline',sumline
                SecProb.linear_constraints.add(lin_expr = sumline, senses =  "E",
                                rhs = [0], names = ["u"+str(i)])
        SecProb.linear_constraints.add(lin_expr = rows, senses = "L",
                                             rhs = [1], names = ["t0"])
    
    my_SecProb = CPX.Cplex()
    handle = populatebyrow(my_SecProb)
    my_SecProb.solve()
    my_SecProb.write("lpex2.lp")
    SecVariables    = my_SecProb.solution.get_values()
    SecObjective=my_SecProb.solution.get_objective_value()
#     print 'SecVariables,SecObjective,my_colnames',SecVariables, SecObjective,my_colnames
    return SecVariables, SecObjective,my_colnames

In [3]:
def GeneratintCutParameters ( m, n, my_rhs, my_coef, BranchedVariables,SecVariables): 
    BranchedLen=len(BranchedVariables)
    unbranched=list()
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)  
    LeUnbranched=len(unbranched)                   
#     NodeNum=n-len(unbranched)
#     k=NodeNum-1 
    lhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),BranchedLen)))
    rhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),1)))

    for i in range (LeUnbranched):   
        for h in range (m):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+m+h][BranchedVariables.index(j)]=my_coef[h][j]*SecVariables[i*(2*m+4*n)+m+h]  
                rhs[i*(2*m+4*n)+m+h]=my_rhs[h]*SecVariables[i*(2*m+4*n)+m+h]  
            
        for h in range (n):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+2*m+2*n+j][BranchedVariables.index(j)]=SecVariables[i*(2*m+4*n)+2*m+2*n+j]  
                rhs[i*(2*m+4*n)+2*m+2*n+h]=SecVariables[i*(2*m+4*n)+2*m+2*n+h]  
            
#         for h in range (n):
        for j in (BranchedVariables):
            lhs[i*(2*m+4*n)+2*m+3*n+j][BranchedVariables.index(j)]=-1*SecVariables[i*(2*m+4*n)+2*m+3*n+j]  

    #     print 'lefttt',lhs,'rightttt', rhs
    LenLeft=len(lhs)
    LenRight=len(rhs)
    LenSec=len(SecVariables)
#     print 'LenLeft',LenLeft
#     print'LenRight',LenRight
#     print 'LenSec',LenSec
#     indicesLhs=[]
#     for j in range (8):
#         indicesLhs.append([i for i in range(LenLeft) if lhs[i][j]!=0]) 
#     print indicesLhs
#     indicesRhs = [(i,rhs[i]) for i in range(LenRight) if rhs[i]>0]
#     print 'indicesRhs',indicesRhs
#     indiceSecVariables=[(i , SecVariables[i]) for i in range(LenSec) if SecVariables[i]!=0]
#     ValueSecVariables=[SecVariables[i] for i in range(LenSec) if SecVariables[i]!=0]
#     print 'positiveLeftHand',indicesLhs 
#     print 'indiceSecVariables',indiceSecVariables
#     print 'ValueSecVariables', ValueSecVariables
    FinalLhs=np.sum(lhs, axis=0)
    FinalRhs=np.sum(rhs, axis=0)
    print 'FinalLhs',FinalLhs, 'FinalRhs',FinalRhs
    return FinalLhs,FinalRhs



In [4]:
class MyCut(CPX_CB.UserCutCallback):

    def __call__(self):
         
        self.times_called += 1
        print  'self.times_called', self.times_called
        branched=list()
        values=list()
        branchedNames=list()
        global BranchedMatrix
        senses= "L"
#         print 'self.get_lower_bounds(s)',self.get_lower_bounds()
#         print 'self.get_upper_bounds(s)',self.get_upper_bounds()
#         print 'objective value', self.get_objective_value()
#         print 'values', self.get_values()
        print 'get_num_nodes', self.get_num_nodes()

        lb=self.get_lower_bounds()
        ub=self.get_upper_bounds()               
        for j in range(len(lb)):
            if lb[j]==ub[j]:
                branched.append(j)
                values.append(lb[j])
                branchedNames.append(VarsName[j])
                
        print 'values',values
        print 'branchedNames',branchedNames
        print 'branched',branched

#         if len(values)>0:  
#             BranchedMatrix.append([branched,values])
# #             BranchedValue.append(values)
# #             print 'BranchedMatrix',BranchedMatrix

#             if [BranchedMatrix[-1]]!=[BranchedMatrix[-2]] :
#                 SecModelParameterOutput=SecModelParameter(values, self.m, self.n, self.my_rhs, self.my_coef,branched)
#                 SecondModelOutput=SecondModel (SecModelParameterOutput[0],SecModelParameterOutput[1],SecModelParameterOutput[2])
#                 print 'SecondModelOutput[1]',SecondModelOutput[1]
#                 print 'SecondModelOutput[0]',SecondModelOutput[0]
#                 if SecondModelOutput[1]>0.000001:
#     #                 print 'bayad cut bezane'
#                     output=GeneratintCutParameters (self.m, self.n, self.my_rhs, self.my_coef,branched, SecondModelOutput[0])
#                     print 'output',output[0]
#                     print 'output[1]',output[1]
#                     lhs=[CPX.SparsePair(ind =branchedNames, val = output[0])]
#                     print 'lhs',lhs[0]
#                     self.add(cut=lhs[0],rhs=output[1][0],sense=senses)
#                     print 'cut added'



In [5]:
def admipex3(c):
    
    c.parameters.preprocessing.presolve = 0

    c.parameters.preprocessing.reduce = 0

    c.parameters.preprocessing.linear.set(0)

    c.parameters.preprocessing.numpass = 0

    c.parameters.mip.strategy.presolvenode = -1

    c.parameters.preprocessing.linear = 0
    
def CutCplex(c):
    c.parameters.mip.cuts.disjunctive.set(-1)
    c.parameters.mip.cuts.bqp.set(-1)
    c.parameters.mip.cuts.cliques.set(-1)
    c.parameters.mip.cuts.covers.set(-1)
    c.parameters.mip.cuts.flowcovers.set(-1)
    c.parameters.mip.cuts.gomory.set(-1)
    c.parameters.mip.cuts.gubcovers.set(-1)
    c.parameters.mip.cuts.implied.set(-1)
    c.parameters.mip.cuts.liftproj.set(-1)
    c.parameters.mip.cuts.localimplied.set(-1)
    c.parameters.mip.cuts.mcfcut.set(-1)
    c.parameters.mip.cuts.mircut.set(-1)
    c.parameters.mip.cuts.zerohalfcut.set(-1)
    c.parameters.mip.cuts.pathcut.set(-1)
    
c = CPX.Cplex('NewExample.lp')
m=c.linear_constraints.get_num()
n=c.variables.get_num()
my_rhs=[]
my_coef=[]
for i in range(m):
    my_row=[]
    for j in range(n):
        my_row.append(c.linear_constraints.get_coefficients(i,j))
    my_coef.append(my_row)

for i in range (m):
    my_rhs.append(c.linear_constraints.get_rhs(i))

admipex3(c)
VarsName=c.variables.get_names()
c.set_log_stream(sys.stdout)
c.set_results_stream(sys.stdout)

cutinst=c.register_callback(MyCut)
cutinst.times_called = 0
cutinst.m=m
cutinst.n=n
cutinst.my_rhs=my_rhs
cutinst.my_coef=my_coef
cutinst.VarsName=VarsName
BranchedMatrix=[[[-1], [-0.0]]]
c.parameters.mip.interval.set(1)
c.parameters.preprocessing.linear=0
c.parameters.mip.strategy.search.set(c.parameters.mip.strategy.search.values.traditional)
c.solve()

c.write("lpex1.lp")
solution = c.solution

# the following line prints the corresponding string
print solution.status[solution.get_status()]
print "Objective value = " , solution.get_objective_value()
print
x = solution.get_values(0, c.variables.get_num()-1)
for j in range(c.variables.get_num()):
    if fabs(x[j]) > 1.0e-10:
        print "Column %d: Value = %17.10g" % (j, x[j])

print "Branch callback was called ", cutinst.times_called, "times"


CPXPARAM_Read_DataCheck                          1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Interval                            1
CPXPARAM_MIP_Strategy_Search                     1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 30 rows, 30 columns, and 896 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.24 ticks)
Probing time = 0.00 sec. (0.04 ticks)
Tried aggregator 1 time.
Reduced MIP has 30 rows, 30 columns, and 896 nonzeros.
Reduced MIP has 30 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.35 ticks)
Probing time = 0.00 sec. (0.04 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.16 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  

     11    13     1374.9801     4     1237.0000     1392.9887      128   12.61%             x16 U     11      9     10
self.times_called 35
get_num_nodes 12
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x3', 'x4', 'x7', 'x8', 'x11', 'x14', 'x15', 'x16', 'x17', 'x19', 'x23', 'x24', 'x25', 'x26', 'x28']
branched [1, 3, 4, 7, 8, 11, 14, 15, 16, 17, 19, 23, 24, 25, 26, 28]
self.times_called 36
get_num_nodes 12
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x3', 'x4', 'x7', 'x8', 'x11', 'x14', 'x15', 'x16', 'x17', 'x19', 'x23', 'x24', 'x25', 'x26', 'x28']
branched [1, 3, 4, 7, 8, 11, 14, 15, 16, 17, 19, 23, 24, 25, 26, 28]
     12    14     1361.9477     5     1237.0000     1392.9887      135   12.61%              x7 U     12     11     11
self.times_called 37
get_num_nodes 13
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1

     26    24     1341.1360     5     1237.0000     1390.2022      190   12.38%             x15 U     26     25      9
self.times_called 62
get_num_nodes 27
values [1.0, 1.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x4', 'x8', 'x11', 'x12', 'x15', 'x16', 'x17', 'x18', 'x19', 'x24', 'x28']
branched [1, 2, 3, 4, 8, 11, 12, 15, 16, 17, 18, 19, 24, 28]
self.times_called 63
get_num_nodes 27
values [1.0, 1.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x4', 'x8', 'x11', 'x12', 'x15', 'x16', 'x17', 'x18', 'x19', 'x24', 'x28']
branched [1, 2, 3, 4, 8, 11, 12, 15, 16, 17, 18, 19, 24, 28]
     27    25     1315.5107     5     1237.0000     1390.2022      197   12.38%             x15 D     27     25      9
self.times_called 64
get_num_nodes 28
values [1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x4', 'x7', 'x8', 'x11',

values [1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x4', 'x7', 'x13', 'x14', 'x15', 'x19', 'x24', 'x25', 'x28']
branched [4, 7, 13, 14, 15, 19, 24, 25, 28]
*    40+   26                         1268.0000     1388.1345             9.47%
     40    28     1338.6725     6     1268.0000     1388.1345      232    9.47%             x14 U     40     39      6
Elapsed time = 0.75 sec. (14.23 ticks, tree = 0.01 MB, solutions = 4)
self.times_called 84
get_num_nodes 41
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x2', 'x3', 'x4', 'x7', 'x8', 'x11', 'x13', 'x14', 'x15', 'x19', 'x20', 'x22', 'x23', 'x24', 'x25', 'x26', 'x28', 'x29']
branched [2, 3, 4, 7, 8, 11, 13, 14, 15, 19, 20, 22, 23, 24, 25, 26, 28, 29]
self.times_called 85
get_num_nodes 41
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x2', 'x3', 'x4', 'x7', 'x8', 'x11', 'x13', 'x14',

self.times_called 106
get_num_nodes 54
values [-0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x2', 'x3', 'x6', 'x7', 'x11', 'x13', 'x14', 'x15', 'x17', 'x18', 'x19', 'x20', 'x22', 'x23', 'x24', 'x25', 'x26', 'x28', 'x29']
branched [0, 2, 3, 6, 7, 11, 13, 14, 15, 17, 18, 19, 20, 22, 23, 24, 25, 26, 28, 29]
     54    35     1284.7474     7     1268.0000     1384.9588      302    9.22%             x18 D     54     52      7
self.times_called 107
get_num_nodes 55
values [-0.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x2', 'x3', 'x5', 'x6', 'x7', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x22', 'x23', 'x24', 'x25', 'x26', 'x28', 'x29']
branched [0, 2, 3, 5, 6, 7, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 28, 29]
self.times_called 108
get_num_nodes 55
values [-0.0, 1.0, 1.0

...

    119    40     1335.0355     8     1317.0000     1358.3555      612    3.14%             x18 U    119    118      7
    120    39    infeasible           1317.0000     1358.3555      612    3.14%              x8 U    120    119      8
Elapsed time = 1.80 sec. (19.66 ticks, tree = 0.01 MB, solutions = 5)
    121    38        cutoff           1317.0000     1358.3555      615    3.14%              x8 D    121    119      8
self.times_called 217
get_num_nodes 122
values [1.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0]
branchedNames ['x2', 'x3', 'x4', 'x7', 'x11', 'x17', 'x19', 'x23', 'x24', 'x26', 'x29']
branched [2, 3, 4, 7, 11, 17, 19, 23, 24, 26, 29]
self.times_called 218
get_num_nodes 122
values [1.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0]
branchedNames ['x2', 'x3', 'x4', 'x7', 'x11', 'x17', 'x19', 'x23', 'x24', 'x26', 'x29']
branched [2, 3, 4, 7, 11, 17, 19, 23, 24, 26, 29]
self.times_called 219
get_num_nodes 122
values [1.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.

    135    41     1334.7784     5     1317.0000     1352.3191      715    2.68%             x26 U    135    134      8
self.times_called 239
get_num_nodes 136
values [1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x4', 'x6', 'x7', 'x8', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x28', 'x29']
branched [1, 2, 3, 4, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29]
self.times_called 240
get_num_nodes 136
values [1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x4', 'x6', 'x7', 'x8', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x28', 'x29']
branched [1, 2, 3, 4, 6, 7, 8, 11, 12, 13, 14, 15, 16,

values [-0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x2', 'x3', 'x5', 'x6', 'x7', 'x11', 'x13', 'x14', 'x15', 'x17', 'x18', 'x19', 'x20', 'x22', 'x23', 'x26', 'x28', 'x29']
branched [0, 1, 2, 3, 5, 6, 7, 11, 13, 14, 15, 17, 18, 19, 20, 22, 23, 26, 28, 29]
self.times_called 258
get_num_nodes 152
values [-0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x2', 'x3', 'x5', 'x6', 'x7', 'x11', 'x13', 'x14', 'x15', 'x17', 'x18', 'x19', 'x20', 'x22', 'x23', 'x26', 'x28', 'x29']
branched [0, 1, 2, 3, 5, 6, 7, 11, 13, 14, 15, 17, 18, 19, 20, 22, 23, 26, 28, 29]
    152    37     1337.8667     8     1317.0000     1346.3080      785    2.23%             x14 U    152    151      7
self.times_called 259
get_num_nodes 153
values [-0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 

    165    34        cutoff           1317.0000     1337.6685      829    1.57%              x4 D    165    163      9
    166    33        cutoff           1317.0000     1337.3982      836    1.55%             x16 U    166     74      6
self.times_called 275
get_num_nodes 167
values [1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x4', 'x6', 'x7', 'x8', 'x11', 'x13', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x23', 'x24', 'x25', 'x26', 'x29']
branched [1, 2, 3, 4, 6, 7, 8, 11, 13, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 29]
    167    32        cutoff           1317.0000     1336.4911      840    1.48%             x26 D    167    134      8
self.times_called 276
get_num_nodes 168
values [1.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x11', 'x14', 'x15', 'x16',

Objective value =  1317.0

Column 1: Value =                 1
Column 2: Value =                 1
Column 3: Value =                 1
Column 4: Value =                 1
Column 6: Value =                 1
Column 7: Value =                 1
Column 8: Value =                 1
Column 11: Value =                 1
Column 12: Value =                 1
Column 13: Value =                 1
Column 14: Value =                 1
Column 15: Value =                 1
Column 16: Value =                 1
Column 18: Value =                 1
Column 20: Value =                 1
Column 22: Value =                 1
Column 23: Value =                 1
Column 25: Value =                 1
Column 28: Value =                 1
Column 29: Value =                 1
Branch callback was called  283 times
